## Real-World Example: N-Body Simulation

Sim£ulate gravitational interactions between particles:

In [ ]:
import numba
import numpy as np
import time
from numba import jit, njit, vectorize, prange
import math

In [ ]:
@njit
def compute_forces(positions, masses, G=1.0, softening=0.01):
    """
    Compute gravitational forces between all particles
    
    Args:
        positions: (N, 2) array of particle positions
        masses: (N,) array of particle masses
        G: gravitational constant
        softening: softening parameter to avoid singularities
    
    Returns:
        forces: (N, 2) array of forces on each particle
    """
    n_particles = len(positions)
    forces = np.zeros_like(positions)
    
    for i in range(n_particles):
        for j in range(i+1, n_particles):
            # Vector from i to j
            dx = positions[j, 0] - positions[i, 0]
            dy = positions[j, 1] - positions[i, 1]
            
            # Distance with softening
            r = math.sqrt(dx*dx + dy*dy + softening*softening)
            
            # Force magnitude
            force_magnitude = G * masses[i] * masses[j] / (r*r*r)
            
            # Force components
            fx = force_magnitude * dx
            fy = force_magnitude * dy
            
            # Newton's 3rd law
            forces[i, 0] += fx
            forces[i, 1] += fy
            forces[j, 0] -= fx
            forces[j, 1] -= fy
    
    return forces

@njit
def update_particles(positions, velocities, masses, dt, G=1.0):
    """Update particle positions and velocities using Euler integration"""
    forces = compute_forces(positions, masses, G)
    
    # Update velocities: v += (F/m) * dt
    for i in range(len(positions)):
        velocities[i, 0] += (forces[i, 0] / masses[i]) * dt
        velocities[i, 1] += (forces[i, 1] / masses[i]) * dt
    
    # Update positions: x += v * dt
    positions += velocities * dt
    
    return positions, velocities

# Create a small solar system
n_particles = 100
np.random.seed(42)

# Random positions in a circle
angles = np.random.random(n_particles) * 2 * np.pi
radii = np.random.random(n_particles) * 5 + 1
positions = np.column_stack([
    radii * np.cos(angles),
    radii * np.sin(angles)
])

# Orbital velocities (perpendicular to radius)
velocities = np.column_stack([
    -radii**(-0.5) * np.sin(angles),
    radii**(-0.5) * np.cos(angles)
]) * 2.0

# Random masses
masses = np.random.random(n_particles) * 0.1 + 0.01
masses[0] = 10.0  # Central massive object
positions[0] = [0, 0]  # Put it at center
velocities[0] = [0, 0]  # At rest

print(f"Simulating {n_particles} particles")

# Time a single step
pos_copy = positions.copy()
vel_copy = velocities.copy()

# Warm up
_ = update_particles(pos_copy, vel_copy, masses, 0.01)

print("Single simulation step:")
%timeit update_particles(pos_copy, vel_copy, masses, 0.01)

# Run a short simulation
n_steps = 1000
dt = 0.01

print(f"\nRunning {n_steps} steps...")
start_time = time.time()

for step in range(n_steps):
    positions, velocities = update_particles(positions, velocities, masses, dt)

end_time = time.time()
print(f"Simulation completed in {end_time - start_time:.2f} seconds")
print(f"Performance: {n_steps / (end_time - start_time):.1f} steps/second")

### Exercise 3: Optimize the N-Body Simulation

The current implementation is O(N²). For large N, this becomes expensive. Try parallelizing it:

In [ ]:
# Exercise 3: Parallel N-body force computation
@njit(parallel=True)
def compute_forces_parallel(positions, masses, G=1.0, softening=0.01):
    """Parallel version of force computation"""
    n_particles = len(positions)
    forces = np.zeros_like(positions)
    
    # TODO: Implement the same logic as in compute_forces
    # Make sure to parallelize the outer loop using prange
    # Be careful about race conditions in force accumulation
    ...
    
    return forces

@njit(parallel=True)
def update_particles_parallel(positions, velocities, masses, dt, G=1.0):
    """Parallel particle update"""
    forces = compute_forces_parallel(positions, masses, G)
    
    # Parallel velocity and position updates
    for i in prange(len(positions)):
        velocities[i, 0] += (forces[i, 0] / masses[i]) * dt
        velocities[i, 1] += (forces[i, 1] / masses[i]) * dt
        positions[i, 0] += velocities[i, 0] * dt
        positions[i, 1] += velocities[i, 1] * dt
    
    return positions, velocities

# Test with larger system
n_particles_large = 200
np.random.seed(42)

# Create larger system
angles = np.random.random(n_particles_large) * 2 * np.pi
radii = np.random.random(n_particles_large) * 5 + 1
positions_large = np.column_stack([
    radii * np.cos(angles),
    radii * np.sin(angles)
])
velocities_large = np.column_stack([
    -radii**(-0.5) * np.sin(angles),
    radii**(-0.5) * np.cos(angles)
]) * 2.0
masses_large = np.random.random(n_particles_large) * 0.1 + 0.01
masses_large[0] = 10.0
positions_large[0] = [0, 0]
velocities_large[0] = [0, 0]

# Make copies for testing
pos1, vel1 = positions_large.copy(), velocities_large.copy()
pos2, vel2 = positions_large.copy(), velocities_large.copy()

# Warm up both versions
_ = update_particles(pos1, vel1, masses_large, 0.01)
_ = update_particles_parallel(pos2, vel2, masses_large, 0.01)

print(f"Performance comparison with {n_particles_large} particles:")
print("Serial version:")
%timeit update_particles(pos1, vel1, masses_large, 0.01)

print("Parallel version:")
%timeit update_particles_parallel(pos2, vel2, masses_large, 0.01)

# Verify results are similar (may have small numerical differences due to parallel execution order)
pos1, vel1 = positions_large.copy(), velocities_large.copy()
pos2, vel2 = positions_large.copy(), velocities_large.copy()

pos1, vel1 = update_particles(pos1, vel1, masses_large, 0.01)
pos2, vel2 = update_particles_parallel(pos2, vel2, masses_large, 0.01)

print(f"\nResults approximately match: {np.allclose(pos1, pos2, rtol=1e-10)}")
if not np.allclose(pos1, pos2, rtol=1e-10):
    print(f"Max difference: {np.max(np.abs(pos1 - pos2)):.2e} (parallel execution order effects)")